#### Fractional Cascading

For the 2-D range search problem, we implemented a 2-level WBST. The first level contains nodes sorted by x-coordinate. Then each internal node contains a secondary WBST which contains all points in the subtree rooted at that internal node and sorted by the y-coordinate. Then construction time for this range tree is $O(n \log n)$, the space consumption is $O(n \log n)$ and the query time is $O(k + \log^2 n)$. To perform a query on the 2-D range $Q = [x_1, x_2] \times [y_1, y_2]$, we first do a 1-D range search over the x-coordinate range $[x_1, x_2]$ to find all candidate secondary trees. Then on each of these candidate secondary trees, we perform a 1-D range search over the y-coordinate range $[y_1, y_2]$ to find all points in $Q$. Now, the important thing to note here is that the 1-D range query on all the candidate secondary trees is over the same range. Furthermore, since candidate secondary trees are rooted at nodes along a given path (paths from $successor(x_1)$ and $predecessor(x_2)$ to some lowest common ancestor), each of the candidate subtrees contain a subset of points from their ancestor's subtrees, so there is a lot of reducnant work being done during the 1-D range search in the secondary trees. We can take advantage of this fact to speed up the query time via `fractional cascading`, which involves maintaining a sorted array inside each node in the first level tree instead of a secondary tree and using pointers that connect elements from different secondary arrays.

Suppose `w` is an internal node in the first level tree. Let `u` and `v` be the left and right children respectively. 
```
     w
   /   \
  u     v 
```

Let $L_y(w)$ be the array of all points in the sub-tree rooted at `w` sorted by y-coordinate. Similarly we have arrays $L_y(u)$ and $L_y(v)$ for nodes `u` and `v` respectively. Now, for each element $p \in L_y(w)$, we create a pointer to the element $q \in L_y(u)$ such that $q = successor(p)$. We can do this in $O(|L_y(w)|)$ by just scanning both arrays left to right in lockstep. Similarly, we also create pointers from $L_y(w)$ elements to their successors in $L_y(v)$. We construct these secondary arrays in a top-down manner, starting from the root node of the first level tree, and then performing in-order (DFS) or level-order (BFS) traversal and constructing the secondary arrays inside visited nodes and creating pointers connecting the elements between each node's secondary array elements and the parent node's secondary array elements. Since the total number of elements across the secondary arrays for all the nodes v with $depth(v) = i$ of the tree is $O(n)$, i.e. $\sum_{v \land depth(v)=i} |L_y(v)| = O(n)$, the cost of constructing all secondary arrays is $O(n \log n)$ since there are at most $\log n$ levels in the tree. The space consumption is also $O(n \log n)$.

To perform a 2-D range query, like before we first identify nodes $[a', b'] =[successor(x_1),predecessor(x_2)]$  and their lowest common ancestor ($LCA$), and then found the paths $L_a$ and $L_b$, i.e. paths from $LCA$ to $a'$ and $b'$ respectively. Then starting at the left child of $LCA$, we just need to perform a single binary search in $L_y(LCA.left)$ to find $successor(y_1)$. If this node $LCA.left$ is a candidate (i.e. if it's $x$ value is $\geq x_1$) we just scan $L_y(LCA.left)$ linearly from $successor(y_1)$ until we reach a point larger than $y_2$, then report all points in that interval. Then we simply walk down one step along the path $L_a$ to a descendent node and use the pointer from $successor(y_1)$ of the parent node to get the successor of that point in the secondary array of this descendent node. If this descendent node is a candidate (i.e. if it's $x$ value is $\geq x_1$), then we again do a linear scan from the successor in the secondary array and report all points found. Thus we keep walking doing the path, following pointers to get the $successor(y_1)$ inside each candicate secondary array and doing a linear scan in that array and reporting the points. We repeat the same thing starting at the right child of $LCA$ and walking down the path $L_b$. 

Note that we are no longer needing to perform binary searches to find $successor(y_1)$ and $predecessor(y_2)$ at every candidate node $v$ on the paths, so the cost of the 1-D range search at each candidate secondary array is $O(k_v)$, where $k_v$ is the number of reported points from the secondary array of that node. Compare this to the case without fractional cascading where we needed to perform binary searches in the secondary tree and the cost of the 1-D range search was $O\Big(k_v + \log(|P(v)|)\Big) = O(k_v +\log n)$. So the total time for the 2-D range query is:

$$O(\log n) + \sum_v O(k_v) = O(k + \log n)$$

where the $O(\log n)$ is from the $O(1)$ binary searches that need to be performed in $L_y(LCA.left)$ and $L_y(LCA.right)$.

